# Text Classification using Neural Networks

<img class=" lazyloaded" data-src="https://bloopglobal.com/wp-content/uploads/2020/11/interactive-presentation-header-1.gif" alt="" title="interactive-presentation-header" src="https://bloopglobal.com/wp-content/uploads/2020/11/interactive-presentation-header-1.gif">

In this project certain classification algorithms will be applied to the given dataset in order to evaluate their performances on different conditions.

In [2]:
 from google.colab import drive
drive.mount("gdrive")

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


## Importing Necessary Libraries and Frameworks

This procedure is done in only one cell for the sake of simplicity and better representation. Sklearn will used in order to train and test different machine learning algorithms on the provided dataset. Pandas will be used in order to obtain observable dataframes and matplotlib.pyplot will be used for plotting operations regarding on the dataset.

In [3]:
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold
from sklearn.metrics import auc,classification_report,RocCurveDisplay,f1_score,accuracy_score,plot_confusion_matrix,confusion_matrix
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import make_pipeline

import gensim.downloader
import nltk
import string 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel

## Importing dataset

In [4]:
PATH = "/content/gdrive/MyDrive/CS-445"

In [5]:
train_df = pd.read_csv(PATH+"/train.csv",index_col=0)
train_df.head(-1)

,id,text,label
0,547530,Beşiktaş'ın eski teknik direktörü Slaven Bilic...,spor
1,42683,"14 Şubat, Katolik Kilisesi’nin azizlerinden St...",yazarlar
2,770925,İstanbul Kağıthane’de gece yarısı sokakta oyun...,video
3,473224,Ziraat Türkiye Kupası maçında deplasmanda Amed...,video
4,44833,"Eski Brezilyalı yıldız oyuncu Ronaldo, Brezily...",spor
...,...,...,...
7994,812733,Çanakkale’den “Herkes İçin Adalet” diye seslen...,yazarlar
7995,786591,"Eskişehirspor Yönetimi, teknik direktörlük gör...",spor
7996,144257,"UEFA Disiplin Kurulu, Borussia Dortmund-Galata...",spor
7997,675701,Rusya Federasyon Konseyi Uluslararası İlişkile...,dunya


In [6]:
test_df = pd.read_csv(PATH+"/test.csv",index_col=0)
test_df.head(-1)

,id,text,label
0,416549,"""Filmimde oynadı ama 32 yıldır görmüyordum abi...",video
1,462801,"Katalonya'nın yeni Başkanı Carles Puigdemont, ...",dunya
2,114173,İstanbul Avcılar'da E-5 üzerinde bir yakıt tan...,video
3,458380,CNN Türk muhabiri emniyet kemeri haberi yapmak...,video
4,287566,\n\n\n \nCumhurbaşkanı Gül yasama yılını açark...,yazarlar
...,...,...,...
1994,51087,"Almanya'nın Köln kentinde, Köln Türkiye Gençli...",video
1995,318095,Turkish Airlines Euroleague'deki temsilcilerim...,spor
1996,283607,Akıl Oyunları filmine ilham veren ünlü matemat...,dunya
1997,244760,Eski ÖSYM Başkanı Ali Demir KPSS soruşturması ...,turkiye


## Metadata about the dataset

###Train Data

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8000 non-null   int64 
 1   text    8000 non-null   object
 2   label   8000 non-null   object
dtypes: int64(1), object(2)
memory usage: 250.0+ KB


In [8]:
train_df.label.value_counts()

turkiye     1630
dunya       1606
yazarlar    1599
spor        1583
video       1582
Name: label, dtype: int64

In [9]:
train_df.isnull().sum(axis = 0)

id       0
text     0
label    0
dtype: int64

###Test Data

In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2000 non-null   int64 
 1   text    2000 non-null   object
 2   label   2000 non-null   object
dtypes: int64(1), object(2)
memory usage: 62.5+ KB


In [11]:
test_df.label.value_counts()

turkiye     421
video       408
dunya       395
yazarlar    392
spor        384
Name: label, dtype: int64

In [12]:
test_df.isnull().sum(axis = 0)

id       0
text     0
label    0
dtype: int64

## Pre-processing

<img width="800" height="400" src="https://thelastbyteblog.files.wordpress.com/2020/06/data-scrubbing-service.gif?w=800" class="attachment-post-thumbnail size-post-thumbnail wp-post-image" alt="" loading="lazy" data-attachment-id="296" data-permalink="https://thelastbyteblog.wordpress.com/2020/06/15/beginners-guide-to-data-preprocessing-in-machine-learning/data-scrubbing-service/" data-orig-file="https://thelastbyteblog.files.wordpress.com/2020/06/data-scrubbing-service.gif" data-orig-size="800,400" data-comments-opened="1" data-image-meta="{&quot;aperture&quot;:&quot;0&quot;,&quot;credit&quot;:&quot;&quot;,&quot;camera&quot;:&quot;&quot;,&quot;caption&quot;:&quot;&quot;,&quot;created_timestamp&quot;:&quot;0&quot;,&quot;copyright&quot;:&quot;&quot;,&quot;focal_length&quot;:&quot;0&quot;,&quot;iso&quot;:&quot;0&quot;,&quot;shutter_speed&quot;:&quot;0&quot;,&quot;title&quot;:&quot;&quot;,&quot;orientation&quot;:&quot;0&quot;}" data-image-title="data-scrubbing-service" data-image-description="" data-medium-file="https://thelastbyteblog.files.wordpress.com/2020/06/data-scrubbing-service.gif?w=300" data-large-file="https://thelastbyteblog.files.wordpress.com/2020/06/data-scrubbing-service.gif?w=750">

There are few options of preprocessing which would allow any ML model to work better. These operations are mainly done on assumptions regarding on the dataset such that the removed words would not have significant effect on identifiying the specific classes.


*   Punctuation removal ✔️
*   Lower casing ✔️
*   Stopwords removal ✔️
*   Stemming or lemmatization (no built-in package) ❌
*   Unnecessary number removal (Capture by regex) ✔️



Samples from the dataset.

In [13]:
for index, row in train_df.iterrows():
    if(index==5):
      break
    print(row[1])
    print("\n")

Beşiktaş'ın eski teknik direktörü Slaven Bilic, Türkiye-Hırvatistan maçında yorumculuk yapmak üzere Lig TV ile anlaştı.
EURO 2016'nın yayıncı kuruluşlarından biri olan Lig TV, Türkiye'nin D Grubu'nda Hırvatistan ile oynayacağı ilk maç için Slaven Bilic ile anlaşıldığını duyurdu.
Beşiktaş'ın eski teknik direktörü Slaven Bilic, 12 Haziran Pazar günü TSİ 16:00'da başlayacak mücadelede yorumcu olacak.
Lig TV, Slaven Bilic'in yanı sıra A Milli Takım'ın efsane kalecilerinden Rüştü Reçber'in de bu karşılaşmanın yorumcularından biri olacağını açıkladı.
EURO 2008'de Hırvatistan'ın teknik direktörü olan Slaven Bilic, çeyrek finalde Türkiye'ye rakip olmuş ve 120 dakikası 1-1 biten maçta A Milli Takımımıza penaltılarda elenmişti.


14 Şubat, Katolik Kilisesi’nin azizlerinden St. Valentine’e adanmış bir gün. “Valentine”in sevgili anlamına gelmesinden olsa gerek “Sevgililer Günü”ne dönüşmüş. 1800’lü yıllarda Amerikalı Esther Howland’ın ilk Sevgililer Günü kartını yollamasıyla yeni bir biçim almış. İ

### Punctuation removal

<img class="img-responsive" src="https://www.thelearninglab.com.sg//content/dam/tll/blog-content/2019/aug/why-proper-punctuation-matters/Why-Proper-Punctuation-Matters_Blog-Header-1_2.gif">

In [14]:
train_df.head(-1)

,id,text,label
0,547530,Beşiktaş'ın eski teknik direktörü Slaven Bilic...,spor
1,42683,"14 Şubat, Katolik Kilisesi’nin azizlerinden St...",yazarlar
2,770925,İstanbul Kağıthane’de gece yarısı sokakta oyun...,video
3,473224,Ziraat Türkiye Kupası maçında deplasmanda Amed...,video
4,44833,"Eski Brezilyalı yıldız oyuncu Ronaldo, Brezily...",spor
...,...,...,...
7994,812733,Çanakkale’den “Herkes İçin Adalet” diye seslen...,yazarlar
7995,786591,"Eskişehirspor Yönetimi, teknik direktörlük gör...",spor
7996,144257,"UEFA Disiplin Kurulu, Borussia Dortmund-Galata...",spor
7997,675701,Rusya Federasyon Konseyi Uluslararası İlişkile...,dunya


In [15]:
train_df["text"]=train_df["text"].str.replace('[{}]'.format(string.punctuation), '')
train_df.head(-1)

,id,text,label
0,547530,Beşiktaşın eski teknik direktörü Slaven Bilic ...,spor
1,42683,14 Şubat Katolik Kilisesi’nin azizlerinden St ...,yazarlar
2,770925,İstanbul Kağıthane’de gece yarısı sokakta oyun...,video
3,473224,Ziraat Türkiye Kupası maçında deplasmanda Amed...,video
4,44833,Eski Brezilyalı yıldız oyuncu Ronaldo Brezilya...,spor
...,...,...,...
7994,812733,Çanakkale’den “Herkes İçin Adalet” diye seslen...,yazarlar
7995,786591,Eskişehirspor Yönetimi teknik direktörlük göre...,spor
7996,144257,UEFA Disiplin Kurulu Borussia DortmundGalatasa...,spor
7997,675701,Rusya Federasyon Konseyi Uluslararası İlişkile...,dunya


In [16]:
test_df["text"]=test_df["text"].str.replace('[{}]'.format(string.punctuation), '')
test_df.head(-1)

,id,text,label
0,416549,Filmimde oynadı ama 32 yıldır görmüyordum abim...,video
1,462801,Katalonyanın yeni Başkanı Carles Puigdemont re...,dunya
2,114173,İstanbul Avcılarda E5 üzerinde bir yakıt tanke...,video
3,458380,CNN Türk muhabiri emniyet kemeri haberi yapmak...,video
4,287566,\n\n\n \nCumhurbaşkanı Gül yasama yılını açark...,yazarlar
...,...,...,...
1994,51087,Almanyanın Köln kentinde Köln Türkiye Gençlik ...,video
1995,318095,Turkish Airlines Euroleaguedeki temsilcilerimi...,spor
1996,283607,Akıl Oyunları filmine ilham veren ünlü matemat...,dunya
1997,244760,Eski ÖSYM Başkanı Ali Demir KPSS soruşturması ...,turkiye


###Lower casing

<img class=" lazyloaded" data-src="https://bloopglobal.com/wp-content/uploads/2020/11/interactive-presentation-header-1.gif" alt="" title="interactive-presentation-header" src="https://i.gifer.com/3VXm.gif">


In [17]:
train_df["text"]=train_df["text"].str.lower()
train_df.head(-1)

,id,text,label
0,547530,beşiktaşın eski teknik direktörü slaven bilic ...,spor
1,42683,14 şubat katolik kilisesi’nin azizlerinden st ...,yazarlar
2,770925,i̇stanbul kağıthane’de gece yarısı sokakta oyu...,video
3,473224,ziraat türkiye kupası maçında deplasmanda amed...,video
4,44833,eski brezilyalı yıldız oyuncu ronaldo brezilya...,spor
...,...,...,...
7994,812733,çanakkale’den “herkes i̇çin adalet” diye sesle...,yazarlar
7995,786591,eskişehirspor yönetimi teknik direktörlük göre...,spor
7996,144257,uefa disiplin kurulu borussia dortmundgalatasa...,spor
7997,675701,rusya federasyon konseyi uluslararası i̇lişkil...,dunya


In [18]:
test_df["text"]=test_df["text"].str.lower()
test_df.head(-1)

,id,text,label
0,416549,filmimde oynadı ama 32 yıldır görmüyordum abim...,video
1,462801,katalonyanın yeni başkanı carles puigdemont re...,dunya
2,114173,i̇stanbul avcılarda e5 üzerinde bir yakıt tank...,video
3,458380,cnn türk muhabiri emniyet kemeri haberi yapmak...,video
4,287566,\n\n\n \ncumhurbaşkanı gül yasama yılını açark...,yazarlar
...,...,...,...
1994,51087,almanyanın köln kentinde köln türkiye gençlik ...,video
1995,318095,turkish airlines euroleaguedeki temsilcilerimi...,spor
1996,283607,akıl oyunları filmine ilham veren ünlü matemat...,dunya
1997,244760,eski ösym başkanı ali demir kpss soruşturması ...,turkiye


###Stopwords removal

<img class=" lazyloaded" data-src="https://bloopglobal.com/wp-content/uploads/2020/11/interactive-presentation-header-1.gif" alt="" title="interactive-presentation-header" src=https://i.gifer.com/UE0b.gif>

In [19]:
stopwords = []
with open("/content/gdrive/MyDrive/CS-445/stopwords-extended") as s_file:
  stopwords = s_file.readlines()

In [20]:
stopwords = [stopword[:-1] for stopword in stopwords]

In [21]:
train_df["text"]= train_df["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [22]:
for index, row in train_df.iterrows():
    if(index==5):
      break
    print(row[1])
    print("\n")

beşiktaşın eski teknik direktörü slaven bilic türkiyehırvatistan maçında yorumculuk lig tv anlaştı euro 2016nın yayıncı kuruluşlarından lig tv türkiyenin d grubunda hırvatistan oynayacağı maç slaven bilic anlaşıldığını duyurdu beşiktaşın eski teknik direktörü slaven bilic 12 haziran pazar günü tsi̇ 1600da başlayacak mücadelede yorumcu olacak lig tv slaven bilicin yanı sıra a milli takımın efsane kalecilerinden rüştü reçberin karşılaşmanın yorumcularından olacağını açıkladı euro 2008de hırvatistanın teknik direktörü slaven bilic çeyrek finalde türkiyeye rakip olmuş 120 dakikası 11 biten maçta a milli takımımıza penaltılarda elenmişti


14 şubat katolik kilisesi’nin azizlerinden st valentine’e adanmış gün “valentine”in sevgili anlamına gelmesinden “sevgililer günü”ne dönüşmüş 1800’lü yıllarda amerikalı esther howland’ın sevgililer günü kartını yollamasıyla yeni biçim almış i̇lk zamanlar sevgililerin birbirlerine kartlarla mesajlar yollamasını özendiren günü sevgililerin geçirmesini sağla

In [23]:
test_df["text"]= test_df["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [24]:
for index, row in test_df.iterrows():
    if(index==5):
      break
    print(row[1])
    print("\n")

filmimde oynadı 32 yıldır görmüyordum abimi diyen heyecanlanarak gözyaşlarına boğulan cem yılmazı izleyenler kahkahalara boğuldu


katalonyanın yeni başkanı carles puigdemont resmen görevi teslim aldığı törende nazım hikmetin güzel şiirini okudu yunan lider çipras başbakanlıktan istifa ederken aynı şiiri okumuştu i̇spanyanın doğusundaki katalonya özerk yönetiminin başkanı seçilen carles puigdemont resmen görevini teslim aldığı törende konuşmada türk şairi nazım hikmetin şiirinden mısrayı okudu katalonyada bağımsızlık yanlısı girişimlerin devam edeceğini vadeden puigdemont bağımsızlık adımlarına atfen fazla şeyleri yaşadığımız uzun seyahatten geliyoruz yorulmadık konuştu puigdemont nazım hikmetin ünlü güzel deniz gidilmemiş olandır mısrasını okudu katalonya özerk yönetiminin 2010 aralık ayından yana başkanı artur masdan başkanlığı devralan puigdemont katalan halkının umudu yaşam koşularının iyileştirilmesi temek haklar kamu hizmeti çevreye duyulan hassasiyetin garanti edilmesidir gerekl

###Number Removal


<img class=" lazyloaded" data-src="https://media.giphy.com/media/5xaOcLwEvFOizxHVyVy/giphy.gif" alt="" title="interactive-presentation-header" src=https://media.giphy.com/media/5xaOcLwEvFOizxHVyVy/giphy.gif>

In [25]:
train_df["text"]=train_df["text"].str.replace(r'\d+', '')

In [26]:
test_df["text"]=test_df["text"].str.replace(r'\d+', '')

## Final modifications before training

In [28]:
x_train = train_df["text"]
y_train = train_df["label"]

In [31]:
nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words("turkish")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Bag-of-words vs Embeddings

In [211]:
import gensim
from itertools import chain

w2v_1 = gensim.models.Word2Vec.load("/content/gdrive/MyDrive/CS-445/project02_word_vec.model")
w2v_2 = gensim.models.KeyedVectors.load_word2vec_format("/content/gdrive/MyDrive/CS-445/trmodel.model",binary=True)
pretrained_weight_1 = w2v_1.wv.vectors
pretrained_weight_2 = w2v_2.wv.vectors

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [32]:
x_test = test_df["text"]
y_test = test_df["label"]

In [217]:
# Importing libraries
from keras.datasets import imdb
from keras.models import Sequential,Model
from keras.layers import Dense,Input,Dropout
from keras.layers import Flatten,GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from keras.layers.merge import concatenate


In [152]:
x_test

0       filmimde oynadı  yıldır görmüyordum abimi diye...
1       katalonyanın yeni başkanı carles puigdemont re...
2       i̇stanbul avcılarda e üzerinde yakıt tankerini...
3       cnn türk muhabiri emniyet kemeri haberi şehirl...
4       \n\n\n cumhurbaşkanı gül yasama yılını açarken...
                              ...                        
1995    turkish airlines euroleaguedeki temsilcilerimi...
1996    akıl oyunları filmine ilham veren ünlü matemat...
1997    eski ösym başkanı ali demir kpss soruşturması ...
1998    atatürk havalimanı katliamından gün milli yas ...
1999    medipol başakşehir kulüp tarihinde şampiyonlar...
Name: text, Length: 2000, dtype: object

In [35]:
all_set_feature=x_train.append(x_test)
all_set_target=y_train.append(y_test)

In [113]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=300)
tokenizer.fit_on_texts(all_set_feature)

In [156]:
Xtrain = tokenizer.texts_to_matrix(all_set_feature[:9000], mode='binary')
Xtest = tokenizer.texts_to_matrix(all_set_feature[9000:], mode='binary')

In [161]:
model = Sequential()
model.add(Input(300,))
model.add(Dense(512,activation='swish'))
model.add(Dense(256,activation='swish'))
model.add(Dense(128,activation='swish'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [162]:
model.fit(Xtrain,pd.get_dummies(all_set_target[:9000]),validation_split=0.1,epochs=10, batch_size=1)

Epoch 1/10
8100/8100 [==============================] - 27s 3ms/step - loss: 0.9015 - accuracy: 0.6632 - val_loss: 0.6621 - val_accuracy: 0.7756
Epoch 2/10
8100/8100 [==============================] - 27s 3ms/step - loss: 0.6366 - accuracy: 0.7755 - val_loss: 0.7048 - val_accuracy: 0.7567
Epoch 3/10
8100/8100 [==============================] - 27s 3ms/step - loss: 0.5454 - accuracy: 0.8054 - val_loss: 0.6212 - val_accuracy: 0.7567
Epoch 4/10
8100/8100 [==============================] - 31s 4ms/step - loss: 0.4740 - accuracy: 0.8317 - val_loss: 0.6663 - val_accuracy: 0.7711
Epoch 5/10
8100/8100 [==============================] - 28s 3ms/step - loss: 0.3955 - accuracy: 0.8575 - val_loss: 0.8122 - val_accuracy: 0.7567
Epoch 6/10
8100/8100 [==============================] - 27s 3ms/step - loss: 0.3454 - accuracy: 0.8845 - val_loss: 0.8570 - val_accuracy: 0.7700
Epoch 7/10
8100/8100 [==============================] - 25s 3ms/step - loss: 0.2635 - accuracy: 0.9142 - val_loss: 0.9163 - val_ac

In [163]:
model.evaluate(Xtest,pd.get_dummies(all_set_target[9000:]))

32/32 [==============================] - 0s 4ms/step - loss: 1.5226 - accuracy: 0.7570


[1.52255117893219, 0.7570000290870667]

### Simple CNN

In [191]:
# Padding the data samples to a maximum review length in words
max_words = 450
X_train = sequence.pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=max_words)
X_test = sequence.pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=max_words)
# Building the CNN Model
model = Sequential()      # initilaizing the Sequential nature for CNN model
# Adding the embedding layer which will take in maximum of 450 words as input and provide a 32 dimensional output of those words which belong in the top_words dictionary
model.add(Embedding(7000, 32, input_length=max_words))
model.add(Conv1D(64, 3, padding='same', activation='swish'))
model.add(MaxPooling1D())
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(256, activation='swish'))
model.add(Dense(128, activation='swish'))
model.add(Dense(5, activation='softmax'))

In [192]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 450, 32)           224000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 450, 64)           6208      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 225, 64)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 225, 32)           6176      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 112, 32)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3584)              0         
_________________________________________________________________
dense_115 (Dense)            (None, 256)             

In [193]:
model.fit(X_train, pd.get_dummies(y_train), validation_split=0.1, epochs=5, batch_size=128, verbose=2)

Epoch 1/5
57/57 - 10s - loss: 1.3282 - accuracy: 0.3988 - val_loss: 1.1916 - val_accuracy: 0.4663
Epoch 2/5
57/57 - 9s - loss: 1.0164 - accuracy: 0.5721 - val_loss: 0.8821 - val_accuracy: 0.6425
Epoch 3/5
57/57 - 10s - loss: 0.7679 - accuracy: 0.6819 - val_loss: 0.8274 - val_accuracy: 0.6637
Epoch 4/5
57/57 - 10s - loss: 0.6889 - accuracy: 0.7303 - val_loss: 0.7057 - val_accuracy: 0.7350
Epoch 5/5
57/57 - 9s - loss: 0.6070 - accuracy: 0.7667 - val_loss: 0.6529 - val_accuracy: 0.7513


In [194]:
model.evaluate(X_test,pd.get_dummies(y_test))

63/63 [==============================] - 1s 14ms/step - loss: 0.6371 - accuracy: 0.7555


[0.6371148228645325, 0.7555000185966492]

These results are not bad and can be evaluated for future studies.

### CNN-rand

In [205]:
def rand_model():

    inputs1 = Input(shape=(450, ))
    embedding1 = Embedding(input_dim=7000,
                           output_dim=32,
                           trainable=True, input_length=450)(inputs1)
    conv1 = Conv1D(filters=16, kernel_size=3, activation='relu'
                   )(embedding1)
    pool1 = MaxPooling1D()(conv1)
    flat1 = Flatten()(pool1)

    
    conv2 = Conv1D(filters=16, kernel_size=4, activation='relu'
                   )(embedding1)
    pool2 = MaxPooling1D()(conv2)
    flat2 = Flatten()(pool2)

    
    conv3 = Conv1D(filters=16, kernel_size=5, activation='relu'
                   )(embedding1)
    pool3 = MaxPooling1D()(conv3)
    flat3 = Flatten()(pool3)
    
    # merge

    merged = concatenate([flat1,flat2,flat3])

    # interpretation
    drop1 = Dropout(0.5)(merged)
    dense1 = Dense(64, activation='relu')(drop1)
    outputs = Dense(5, activation='softmax')(dense1)
    model = Model(inputs=[inputs1], outputs=outputs)

    # compile

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    # summarize

    print(model.summary())
    return model
model = rand_model()
model.fit(X_train, pd.get_dummies(y_train), validation_split=0.1, epochs=5, batch_size=128, verbose=2)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 450)]        0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 450, 32)      224000      input_31[0][0]                   
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 448, 16)      1552        embedding_10[0][0]               
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 447, 16)      2064        embedding_10[0][0]               
______________________________________________________________________________________________

In [206]:
model.evaluate(X_test,pd.get_dummies(y_test))

63/63 [==============================] - 1s 13ms/step - loss: 0.5929 - accuracy: 0.7805


[0.592906653881073, 0.7804999947547913]

### CNN-static

In [218]:
def static_model():

    # channel 1

    inputs1 = Input(shape=(450, ))
    embedding1 = Embedding(input_dim=pretrained_weight_2.shape[0],
                           output_dim=400,
                           weights=[pretrained_weight_2],
                           trainable=False, input_length=450)(inputs1)
    conv1 = Conv1D(filters=16, kernel_size=2, activation='relu'
                   )(embedding1)
    pool1 = GlobalMaxPooling1D()(conv1)

    flat1 = Flatten()(pool1)

    
    conv2 = Conv1D(filters=16, kernel_size=3, activation='relu'
                   )(embedding1)
    pool2 = GlobalMaxPooling1D()(conv2)
    flat2 = Flatten()(pool2)

    
    conv3 = Conv1D(filters=16, kernel_size=4, activation='relu'
                   )(embedding1)
    pool3 = GlobalMaxPooling1D()(conv3)

    flat3 = Flatten()(pool3)

    # merge

    merged = concatenate([flat1,flat2,flat3])

    # interpretation
    dense = Dense(32, activation='relu')(merged)
    drop1 = Dropout(0.2)(dense)
    outputs = Dense(5, activation='softmax')(drop1)
    model = Model(inputs=[inputs1], outputs=outputs)

    # compile

    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

    # summarize

    print(model.summary())
    return model
model = static_model()
model.fit(X_train, pd.get_dummies(y_train), validation_split=0.1, epochs=5, batch_size=128, verbose=2)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           [(None, 450)]        0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 450, 400)     164982800   input_36[0][0]                   
__________________________________________________________________________________________________
conv1d_24 (Conv1D)              (None, 449, 16)      12816       embedding_13[0][0]               
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 448, 16)      19216       embedding_13[0][0]               
____________________________________________________________________________________________

In [219]:
model.evaluate(X_test,pd.get_dummies(y_test))

63/63 [==============================] - 4s 67ms/step - loss: 0.7283 - accuracy: 0.7235


[0.7282633185386658, 0.7235000133514404]

### CNN-non-static

In [224]:
def non_static_model():

    # channel 1

    inputs1 = Input(shape=(450, ))
    embedding1 = Embedding(input_dim=pretrained_weight_2.shape[0],
                           output_dim=400,
                           weights=[pretrained_weight_2],
                           trainable=True, input_length=450)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=2, activation='relu'
                   )(embedding1)
    pool1 = GlobalMaxPooling1D()(conv1)
    flat1 = Flatten()(pool1)

    
    conv2 = Conv1D(filters=32, kernel_size=3, activation='relu'
                   )(embedding1)
    pool2 = GlobalMaxPooling1D()(conv2)
    flat2 = Flatten()(pool2)

    
    conv3 = Conv1D(filters=32, kernel_size=4, activation='relu'
                   )(embedding1)
    pool3 = GlobalMaxPooling1D()(conv3)
    flat3 = Flatten()(pool3)

    # merge

    merged = concatenate([flat1,flat2,flat3])

    # interpretation
    dense = Dense(32, activation='relu')(merged)
    drop1 = Dropout(0.2)(dense)
    outputs = Dense(5, activation='softmax')(drop1)
    model = Model(inputs=[inputs1], outputs=outputs)

    # compile

    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

    # summarize

    print(model.summary())
    return model
model = non_static_model()
model.fit(X_train, pd.get_dummies(y_train), validation_split=0.1, epochs=5, batch_size=128, verbose=2)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           [(None, 450)]        0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 450, 400)     164982800   input_39[0][0]                   
__________________________________________________________________________________________________
conv1d_30 (Conv1D)              (None, 449, 32)      25632       embedding_15[0][0]               
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 448, 32)      38432       embedding_15[0][0]               
____________________________________________________________________________________________

In [226]:
model.evaluate(X_test,pd.get_dummies(y_test))

63/63 [==============================] - 6s 96ms/step - loss: 0.7021 - accuracy: 0.7485


[0.7020707130432129, 0.7484999895095825]